## Problem 1 - Transient Conduction in a Plane Wall

The problem we consider is that of a plane wall initially at 100$^\circ$C with its outer surfaces exposed to an ambient temperature of 0$^\circ$C.  The wall has a thickness 2*L* and may be considered to have an infinite height and a unit depth. The domain is shown schematically below:

![PlaneWall](Figures/2-PlaneWall.png)

Initially, the wall only feels the effect of the ambient air very near the surface and thus, the temperature profile inside the solid is quite steep in the vicinity of the surface.  An analytical solution for this problem involves several (at least 4) terms of a Fourier series.  After some time, however, the influence of the ambient air will have reached the center of the wall and the analytical solution can be approximated by the first term of the Fourier series (see, for example, *Fundamentals of Heat and Mass Transfer* by Incropera et al.). To study the order of accuracy of the fully implicit first and second order time discretization schemes, we will consider the cooling process during a period past the initial transient where the one-term Fourier solution is valid.

The parameters for the problem are:

$$ Bi = \frac{h L}{k}= 1.0 $$

$$ T_i = 100^\circ C $$

$$ T_{\infty}= 0^\circ C $$

The one-term Fourier solution for this problem is:

$$
\frac{T-T_{\infty}}{T_i-T_{\infty}}=C_1 \exp\left(-\zeta^2\frac{\alpha t}{L^2}\right)\cos\left(\zeta \frac{x}{L}\right)
$$

where:

$$ T = T(x,t) $$

$$ \alpha = \frac{k}{\rho c_p} $$

$$ C_1 = 1.1191 $$

$$ \zeta = 0.8603 $$

The solution to this problem at the two different dimensionless time levels of interest is:

$$ \text{at } \frac{\alpha t_1}{L^2}= 0.4535,~~~ T(0,t_1)= 80^\circ C $$

$$ \text{at } \frac{\alpha t_2}{L^2}= 3.2632,~~~ T(0,t_2)= 10^\circ C $$

To solve this problem, initialize the temperature field using the analytical solution at $\alpha t_1/L^2 = 0.4535$. This avoids the need for a very small timestep during the initial transient when solution is changing rapidly. Then, use your code to calculate the temporal variation of the temperature field over the time period described above.  Solve the problem by employing 2, 4, 8, 16, and 32 time steps using both the first and second order implicit schemes.

At the end of each run, calculate the absolute average error, $\overline{e}$, using the formula:

$$
\overline{e}= \frac{1}{N_{CV}} \sum_{i=1}^{N_{CV}} |e(i)|
$$

where

$$ e(i) = T_{exact}(i) - T(i) $$ 

Then, for each scheme, plot your results of $\overline{e}$ vs. $\Delta t$ (on a log-log scale) and find the value of $p$ in the expression:

$$
\overline{e}= c (\Delta t)^p
$$

where $p$ represents the order accuracy of the transient scheme. Also show a separate plot of T(0,$t_2$) verses the number of timesteps used for each scheme employed.

Repeat this problem on at least three different grids to demonstrate grid independence of the solution.

**Bonus**: Solve the same problem using the Crank-Nicolson scheme and compare the results.



# SOLUTION

In [ ]:
class RobinBc:
    """Class defining a Robinn boundary condition"""
   
    def __init__(self, phi, grid, h, k, To, loc):
        """Constructor
            phi ........ field variable array
            grid ....... grid
            h........... convection coefficient
            k........... thermal conductivity
            Tamb........ ambient temperature
            loc ........ boundary location
        """
        self._phi = phi
        self._grid = grid
        self._h = h
        self._k = k
        self._To = To
        self._loc = loc
       
    def value(self):
        """Return the boundary condition value"""
        if self._loc is BoundaryLocation.WEST:
            return (self._phi[1] + ((self._h/self._k)*self._grid.dx_WP[0]* self._To))/(1+((self._h/self._k)*self._grid.dx_WP[0]))
        elif self._loc is BoundaryLocation.EAST:
            return (self._phi[-2] + ((self._h/self._k)*self._grid.dx_PE[-1]* self._To))/(1+((self._h/self._k)*self._grid.dx_PE[-1]))
        else:
            raise ValueError("Unknown boundary location")
   
    def coeff(self):
        """Return the linearization coefficient"""
        if self._loc is BoundaryLocation.WEST:
            return (1/(1+((self._h/self._k)*self._grid.dx_WP[0])))
        elif self._loc is BoundaryLocation.EAST:
            return (1/(1+((self._h/self._k)*self._grid.dx_PE[-1])))
     
    def apply(self):
        """Applies the boundary condition in the referenced field variable array"""
        if self._loc is BoundaryLocation.WEST:
            self._phi[0] = (self._phi[1] + ((self._h/self._k)*self._grid.dx_WP[0]* self._To))/(1+((self._h/self._k)*self._grid.dx_WP[0]))
        elif self._loc is BoundaryLocation.EAST:
            self._phi[-1] = (self._phi[-2] + ((self._h/self._k)*self._grid.dx_PE[-1]* self._To))/(1+((self._h/self._k)*self._grid.dx_PE[-1]))
        else:
            raise ValueError("Unknown boundary location")
            

In [ ]:
class FirstOrderTransientModel:
    """Class defining a first order implicit transient model"""

    def __init__(self, grid, T, Told, rho, cp, dt):
        """Constructor"""
        self._grid = grid
        self._T = T
        self._Told = Told
        self._rho = rho
        self._cp = cp
        self._dt = dt

    def add(self, coeffs):
        """Function to add transient term to coefficient arrays"""

        # Calculate the transient term
        
        trans = (self._rho*self._cp*self._grid.vol)*((self._T[1:-1] - self._Told[1:-1])/self._dt)
        
        # Calculate the linearization coefficient
        
        coeffPt = (self._rho*self._cp*self._grid.vol)/self._dt
        
        # Add to coefficient arrays
        
        coeffs.accumulate_aP(coeffPt)
        coeffs.accumulate_rP(trans)

        return coeffs

In [ ]:
class SecondOrderTransientModel:
    """Class defining a first order implicit transient model"""

    def __init__(self, grid, T, Told, Told2, rho, cp, dt):
        """Constructor"""
        self._grid = grid
        self._T = T
        self._Told = Told
        self._Told2 = Told2
        self._rho = rho
        self._cp = cp
        self._dt = dt

    def add(self, coeffs, tStep):
        """Function to add transient term to coefficient arrays"""

        if tStep == 0:
            
            #Calculate the transient term
            trans = (self._rho*self._cp*self._grid.vol)*((self._T[1:-1] - self._Told[1:-1])/self._dt)
            
            #Calculate the linearization coefficient
            coeffPt = (self._rho*self._cp*self._grid.vol)/self._dt
        else:
            
            #Calculate the transient term
            trans = (self._rho*self._cp*self._grid.vol)*(((3*self._T[1:-1]/2) - (2*self._Told[1:-1]) + (self._Told2 [1:-1]/2))/self._dt)
            
            #Calculate the linearization coefficient
            coeffPt = (3*self._rho*self._cp*self._grid.vol)/(2*self._dt)
                  
        # Add to coefficient arrays
        
        coeffs.accumulate_aP(coeffPt)
        coeffs.accumulate_rP(trans)

        return coeffs

## Grid Independence Test

This grid independence test is being done on the solution of the problem using the first order implicit method for the transient term

### Number of control values : 10

In [ ]:
import math
import numpy as np
from numpy.linalg import norm
import math

from Classes.Grid import Grid
from Classes.ScalarCoeffs import ScalarCoeffs
from Classes.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Classes.Models import DiffusionModel, SurfaceConvectionModel
from Classes.LinearSolver import solve


# Define the grid
lx = 1.0
ly = 0.1
lz = 0.1
ncv = 10
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
dt = 0.087803
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 100
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
To = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

#Analytical temperature for initial condition

C1 = 1.1191
z = 0.8603
Ti = 100

T = np.ones(grid.ncv+2)
for i in range(ncv+2):
    T[i] = ((Ti-To)*((1.1191*np.exp((-z**2)*time))*np.cos((z*grid.xP[i])/lx))) + To

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0 , BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, ho, k, To, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)


# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    
print(T_solns[-1])

### For a number of control volumes = 40

In [ ]:

# Define the grid
lx = 1.0
ly = 0.1
lz = 0.1
ncv = 40
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
dt = 0.087803
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 100
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
To = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

#Analytical temperature for initial condition

C1 = 1.1191
z = 0.8603
Ti = 100

T = np.ones(grid.ncv+2)
for i in range(ncv+2):
    T[i] = ((Ti-To)*((1.1191*np.exp((-z**2)*time))*np.cos((z*grid.xP[i])/lx))) + To

# Define boundary conditions
west_bc = NeumannBc(T, grid, 0 , BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, ho, k, To, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
T_solns = [np.copy(T)]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)


# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    
print(T_solns[-1])

### For a number of control volumes = 20

In [ ]:
import math
import numpy as np
from numpy.linalg import norm
import math

from Classes.Grid import Grid
from Classes.ScalarCoeffs import ScalarCoeffs
from Classes.BoundaryConditions import BoundaryLocation, DirichletBc, NeumannBc
from Classes.Models import DiffusionModel, SurfaceConvectionModel
from Classes.LinearSolver import solve

# Define the grid
lx = 1.0
ly = 0.1
lz = 0.1
ncv = 20
grid = Grid(lx, ly, lz, ncv)

# Set the timestep information
nTime = 32
totalT = 3.2632 - 0.4535
dt = totalT/nTime
time = 0.4535

# Set the maximum number of iterations and convergence criterion
maxIter = 100
converged = 1e-6

# Define thermophysical properties
rho = 1
cp = 1
k = 1

# Define convection parameters
ho = 1
To = 0

# Define the coefficients
coeffs = ScalarCoeffs(grid.ncv)

#Analytical temperature for initial condition

C1 = 1.1191
z = 0.8603
Ti = 100

T = np.ones(grid.ncv+2)
for i in range(ncv+2):
    T[i] = ((Ti-To)*((1.1191*np.exp((-z**2)*time))*np.cos((z*grid.xP[i])/lx))) + To


# Define boundary conditions
west_bc = NeumannBc(T, grid, 0 , BoundaryLocation.WEST)
east_bc = RobinBc(T, grid, ho, k, To, BoundaryLocation.EAST)

# Apply boundary conditions
west_bc.apply()
east_bc.apply()

# Create list to store the solutions at each timestep
T_solns = [np.copy(T)]
T_cent = [T[0]]

# Define the transient model
Told = np.copy(T)
transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

# Define the diffusion model
diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)


# Loop through all timesteps
for tStep in range(nTime):
    # Update the time information
    time += dt
    
    # Print the timestep information
    print("Timestep = {}; Time = {}".format(tStep, time))
    
    # Store the "old" temperature field
    Told[:] = T[:]
    
    # Iterate until the solution is converged
    for i in range(maxIter):
        # Zero the coefficients and add each influence
        coeffs.zero()
        coeffs = diffusion.add(coeffs)
        coeffs = transient.add(coeffs)

        # Compute residual and check for convergence 
        maxResid = norm(coeffs.rP, np.inf)
        avgResid = np.mean(np.absolute(coeffs.rP))
        print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
        if maxResid < converged:
            break
    
        # Solve the sparse matrix system
        dT = solve(coeffs)
    
        # Update the solution and boundary conditions
        T[1:-1] += dT
        west_bc.apply()
        east_bc.apply()
    
    # Store the solution
    T_solns.append(np.copy(T))
    T_cent.append(T[0])

T_exp32 = T_solns[-1]
dt32 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent32 = np.copy(T_cent)
deltat32 = np.linspace(0.4535, 3.2632, 33)

#To monitor the result obtained
print(T_solns[-1])


## Profile temperatures for a number of control volumes of 20 and a number of time step of 32

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

i = 0
for T in T_solns:
    plt.plot(grid.xP, T, label=str(i))
    i += 1

plt.xlabel("x [m]")
plt.ylabel("T [C] ")
plt.legend()
plt.show()

## Analytical solution for the final time

In [ ]:
#Definition of the variables
t2 = 3.2632

#Analytical temperature for initial condition

C1 = 1.1191
z = 0.8603
Ti = 100

T_final = np.ones(grid.ncv+2)
for i in range(ncv+2):
    T_final[i] = ((Ti-To)*((1.1191*np.exp((-z**2)*t2))*np.cos((z*grid.xP[i])/lx))) + To

print (T_final)



### Grid independence analysis

##### For a number of control values of 40

[10.66804431 10.66804431 10.66310951 10.65324219 10.63844691 10.61873053
 10.59410216 10.56457319 10.53015729 10.49087036 10.4467306  10.3977584
 10.34397644 10.28540958 10.22208492 10.15403175 10.08128154 10.00386796
  9.92182682  9.83519605  9.74401575  9.64832807  9.5481773   9.44360975
  9.33467379  9.22141982  9.10390023  8.98216938  8.85628357  8.72630104
  8.59228192  8.4542882   8.31238371  8.1666341   8.01710679  7.86387094
  7.70699743  7.54655884  7.38262938  7.21528488  7.04460275  6.95763235]


#### For a number of control values of 30

[10.66861809 10.66861809 10.6598451  10.64230632 10.61601618 10.58099631
 10.53727549 10.48488967 10.42388195 10.35430248 10.27620848 10.18966418
 10.09474072  9.99151619  9.88007545  9.76051014  9.63291859  9.49740573
  9.35408297  9.20306819  9.04448555  8.87846548  8.70514448  8.52466508
  8.3371757   8.14283052  7.94178933  7.73421747  7.52028563  7.30016972
  7.07405075  6.9580827 ]

#### For a number of control values of 20 this is the profile temperature

[10.67025731 10.67025731 10.65051796 10.61107578 10.55200374 10.4734111
 10.37544328 10.25828149 10.12214248  9.96727811  9.79397486  9.60255333
  9.39336764  9.16680478  8.92328386  8.66325539  8.38720041  8.09562961
  7.78908236  7.46812577  7.13335359  6.95936936]
  
#### For a number of control values of 10:

[10.67910454 10.67910454 10.60014488 10.44280937 10.20826132  9.89823496
  9.51502256  9.06145754  8.54089349  7.95717938  7.31463111  6.96631534]
  
#### For a number of control values of 5:

[10.71441655 10.71441655 10.39854528  9.77611492  8.86547532  7.69347301
  6.99406637]
  


#### From the previous results, without doing the GCI test on this problem, is possible to notice that after the number of control values of 20,  the variance of the results is smaller making the grid independent of the results. The profile temperatures for the last time step was the value of reference to compare between the different number of control volumes, the temperature at x=0 was the one used to compare and its variation.
#### In for this problem, a number of control volumes of 20 will be used to be compared with the other methods.

# First order implicit scheme

## Definition of first order implicit scheme function for calculation

In [ ]:
def firstorder (nTime):

    # Define the grid
    lx = 1.0
    ly = 0.1
    lz = 0.1
    ncv = 20
    grid = Grid(lx, ly, lz, ncv)

    # Set the timestep information
    totalT = 3.2632 - 0.4535
    dt = totalT/nTime
    time = 0.4535

    # Set the maximum number of iterations and convergence criterion
    maxIter = 100
    converged = 1e-6

    # Define thermophysical properties
    rho = 1
    cp = 1
    k = 1

    # Define convection parameters
    ho = 1
    To = 0

    # Define the coefficients
    coeffs = ScalarCoeffs(grid.ncv)

    #Analytical temperature for initial condition

    C1 = 1.1191
    z = 0.8603
    Ti = 100

    T = np.ones(grid.ncv+2)
    for i in range(ncv+2):
        T[i] = ((Ti-To)*((1.1191*np.exp((-z**2)*time))*np.cos((z*grid.xP[i])/lx))) + To

    # Define boundary conditions
    west_bc = NeumannBc(T, grid, 0 , BoundaryLocation.WEST)
    east_bc = RobinBc(T, grid, ho, k, To, BoundaryLocation.EAST)

    # Apply boundary conditions
    west_bc.apply()
    east_bc.apply()

    # Create list to store the solutions at each timestep
    T_solns = [np.copy(T)]
    T_cent = [T[0]]

    # Define the transient model
    Told = np.copy(T)
    transient = FirstOrderTransientModel(grid, T, Told, rho, cp, dt)

    # Define the diffusion model
    diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)


    # Loop through all timesteps
    for tStep in range(nTime):
        # Update the time information
        time += dt

        # Print the timestep information
        print("Timestep = {}; Time = {}".format(tStep, time))

        # Store the "old" temperature field
        Told[:] = T[:]

        # Iterate until the solution is converged
        for i in range(maxIter):
            # Zero the coefficients and add each influence
            coeffs.zero()
            coeffs = diffusion.add(coeffs)
            coeffs = transient.add(coeffs)

            # Compute residual and check for convergence 
            maxResid = norm(coeffs.rP, np.inf)
            avgResid = np.mean(np.absolute(coeffs.rP))
            print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
            if maxResid < converged:
                break

            # Solve the sparse matrix system
            dT = solve(coeffs)

            # Update the solution and boundary conditions
            T[1:-1] += dT
            west_bc.apply()
            east_bc.apply()

        # Store the solution
        T_solns.append(np.copy(T))
        T_cent.append(T[0])
        
    return (T_solns, T_cent, dt)


## Number of time step  = 2

In [ ]:
T_solns2, T_cent2, dt = firstorder(2)

T_exp2 = T_solns2[-1]
dt2 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent2 = np.copy(T_cent2)
print(T_cent2)
deltat2 = np.linspace(0.4535, 3.2632, 3)
print(deltat2)

#To show the solution temperature for verification
print(T_solns2[-1])

### Calculation of the error for a time step of 2

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp2[1:-1]
print(err)
sum_err = np.sum(err)

err_bar2 = np.abs(sum_err)/grid.ncv

print (err_bar2)

## Number of time step of 4

In [ ]:
T_solns4, T_cent4, dt = firstorder(4)

T_exp4 = T_solns4[-1]
dt4 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent4 = np.copy(T_cent4)
print(T_cent4)
deltat4 = np.linspace(0.4535, 3.2632, 5)
print(deltat4)

#To show the solution temperature for verification
print(T_solns4[-1])

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp4[1:-1]
print(err)
sum_err = np.sum(err)

err_bar4 = np.abs(sum_err)/grid.ncv

print (err_bar4)

### Number of time step of 8

In [ ]:
T_solns8, T_cent8, dt = firstorder(8)

T_exp8 = T_solns8[-1]
dt8 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent8 = np.copy(T_cent8)
print(T_cent8)
deltat8 = np.linspace(0.4535, 3.2632, 9)
print(deltat8)

#To show the solution temperature for verification
print(T_solns8[-1])

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp8[1:-1]
print(err)
sum_err = np.sum(err)

err_bar8 = np.abs(sum_err)/grid.ncv

print (err_bar8)

### Number of time step of 16

In [ ]:
T_solns16, T_cent16, dt = firstorder(16)

T_exp16 = T_solns16[-1]
dt16 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent16 = np.copy(T_cent16)
print(T_cent16)
deltat16 = np.linspace(0.4535, 3.2632, 17)
print(deltat16)

#To show the solution temperature for verification
print(T_solns16[-1])

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp16[1:-1]
print(err)
sum_err = np.sum(err)

err_bar16 = np.abs(sum_err)/grid.ncv

print (err_bar16)

### Calculation of the error for a Time step of 32

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp32[1:-1]
print(err)
sum_err = np.sum(err)

err_bar32 = np.abs(sum_err)/grid.ncv

print (err_bar32)

## Calculation of the value of P

Using the errors calculated for each time step, the value of P is founded with the following procedure. Since the value of p represents the order accuracy of the transient scheme, is expected that this value is close to 1.

In [ ]:
#To generate the plot of e bar vs deltat

from scipy import stats
import numpy as np

x = np.array([dt2, dt4, dt8, dt16, dt32])
print (x)
y = np.array([err_bar2, err_bar4, err_bar8, err_bar16, err_bar32])

print (y)

res = stats.linregress(np.log(x), np.log(y))

#slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
print(res.slope)
print ("Value of P is:", res.slope)

plt.plot(np.log(x), np.log(y), 'o', label='original data')
plt.plot(x, res.intercept + res.slope*x, 'r', label='fitted line')

### Analysis of the results of the value of P

As expected, the value of P is close to 1, following the order of this scheme. Is possible to notice that the error decreases by the double with the increase of the number of time step as we might expect for this implicit first order scheme method. In this case is possible to notice that for the number of time step of 32 is still high since it is approximate 0.60, if the problem to be solved needs a more accurate result with a small percentage of error, is important to increase the number of time step until obtain the error desired. 

## Plot of the changes of the temperature at x=0 at the different time step

In order to analyze the behaviour of the method, the following plot represents the temperature at the centre of the wall and it change with time for the different number of time step.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig1, ax = plt.subplots()


ax.plot(deltat2, T_cent2, label="N time step = 2")
ax.plot(deltat4, T_cent4, label="N time step = 4")
ax.plot(deltat8, T_cent8, label="N time step = 8")
ax.plot(deltat16, T_cent16, label="N time step = 16")
ax.plot(deltat32, T_cent32, label="N time step = 32")


ax.set_title("T (0,t2) vs delta T for a first order implicit scheme")
ax.set_xlabel("Delta t [s]")
ax.set_ylabel("T [C]")
ax.legend()
plt.show()

## Analysis

As discussed before with the error, the higher number of time step provide a smoother curve in the results of the temperature disminution along the defined time. The smaller the time step the more accurate result within time since it is a first order scheme.

# Second order implicit scheme

## Definition of the function for the second order implicit scheme 

In [ ]:
import math

def secondorder (nTime):
    # Define the grid
    lx = 1.0
    ly = 0.1
    lz = 0.1
    ncv = 20
    grid = Grid(lx, ly, lz, ncv)

    # Set the timestep information
    totalT = 3.2632 - 0.4535
    dt = totalT/nTime
    time = 0.4535

    # Set the maximum number of iterations and convergence criterion
    maxIter = 100
    converged = 1e-6

    # Define thermophysical properties
    rho = 1
    cp = 1
    k = 1

    # Define convection parameters
    ho = 1
    To = 0

    # Define the coefficients
    coeffs = ScalarCoeffs(grid.ncv)

    #Analytical temperature for initial condition

    C1 = 1.1191
    z = 0.8603
    Ti = 100

    T = np.ones(grid.ncv+2)
    for i in range(ncv+2):
        T[i] = ((Ti-To)*((1.1191*np.exp((-z**2)*time))*np.cos((z*grid.xP[i])/lx))) + To


    #Define boundary conditions
    west_bc = NeumannBc(T, grid, 0 , BoundaryLocation.WEST)
    east_bc = RobinBc(T, grid, ho, k, To, BoundaryLocation.EAST)

    # Apply boundary conditions
    west_bc.apply()
    east_bc.apply()

    # Create list to store the solutions at each timestep
    T_solns = [np.copy(T)]
    T_cent = [T[0]]


    # Define the transient model
    Told = np.copy(T)
    Told2 = np.copy(Told) 
    transient = SecondOrderTransientModel(grid, T, Told, Told2, rho, cp, dt)

    # Define the diffusion model
    diffusion = DiffusionModel(grid, T, k, west_bc, east_bc)

    # Loop through all timesteps
    for tStep in range(nTime):
        # Update the time information
        time += dt

        # Print the timestep information
        print("Timestep = {}; Time = {}".format(tStep, time))

        # Store the "old" temperature field
        Told2[:] = Told[:]
        Told[:] = T[:]

        # Iterate until the solution is converged
        for i in range(maxIter):
            # Zero the coefficients and add each influence
            coeffs.zero()
            coeffs = diffusion.add(coeffs)
            coeffs = transient.add(coeffs, tStep)

            # Compute residual and check for convergence 
            maxResid = norm(coeffs.rP, np.inf)
            avgResid = np.mean(np.absolute(coeffs.rP))
            print("Iteration = {}; Max. Resid. = {}; Avg. Resid. = {}".format(i, maxResid, avgResid))
            if maxResid < converged:
                break

            # Solve the sparse matrix system
            dT = solve(coeffs)

            # Update the solution and boundary conditions
            T[1:-1] += dT
            west_bc.apply()
            east_bc.apply()

        # Store the solution
        T_solns.append(np.copy(T))
        T_cent.append(T[0])

    return (T_solns, T_cent, dt)


## Number of time step = 32

In [ ]:
T_solns32, T_cent32, dt = secondorder(32)

T_exp32 = T_solns32[-1]
dt32 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent32 = np.copy(T_cent32)
print(T_cent32)
deltat32 = np.linspace(0.4535, 3.2632, 33)
print(deltat32)

#To show the solution temperature for verification
print(T_solns32[-1])


### Calculation of the error

In [ ]:
err = T_final[1:-1] - T_exp32[1:-1]
print(err)
sum_err = np.sum(err)

err_bar32 = np.abs(sum_err)/grid.ncv

print (err_bar32)

## Number of time step = 2

In [ ]:
T_solns2, T_cent2, dt = secondorder(2)

T_exp2 = T_solns2[-1]
dt2 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent2 = np.copy(T_cent2)
print(T_cent2)
deltat2 = np.linspace(0.4535, 3.2632, 3)
print(deltat2)

#To show the solution temperature for verification
print(T_solns2[-1])



### Calculation of the error

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp2[1:-1]
print(err)
sum_err = np.sum(err)

err_bar2 = np.abs(sum_err)/grid.ncv

print (err_bar2)

## Number of time step = 4

In [ ]:
T_solns4, T_cent4, dt = secondorder(4)

T_exp4 = T_solns4[-1]
dt4 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent4 = np.copy(T_cent4)
print(T_cent4)
deltat4 = np.linspace(0.4535, 3.2632, 5)
print(deltat4)

#To show the solution temperature for verification
print(T_solns4[-1])



### Calculation of the error

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp4[1:-1]
print(err)
sum_err = np.sum(err)

err_bar4 = np.abs(sum_err)/grid.ncv

print (err_bar4)

## Number of time step = 8

In [ ]:
T_solns8, T_cent8, dt = secondorder(8)

T_exp8 = T_solns8[-1]
dt8 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent8 = np.copy(T_cent8)
print(T_cent8)
deltat8 = np.linspace(0.4535, 3.2632, 9)
print(deltat8)

#To show the solution temperature for verification
print(T_solns8[-1])



### Calculation of the error

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp8[1:-1]
print(err)
sum_err = np.sum(err)

err_bar8 = np.abs(sum_err)/grid.ncv

print (err_bar8)

## Number of time step = 16

In [ ]:
T_solns16, T_cent16, dt = secondorder(16)

T_exp16 = T_solns16[-1]
dt16 = dt

#Store the temperature at x=0 and create the x values for deltat
T_cent16 = np.copy(T_cent16)
print(T_cent16)
deltat16 = np.linspace(0.4535, 3.2632, 17)
print(deltat16)

#To show the solution temperature for verification
print(T_solns16[-1])



### Calculation of the error

In [ ]:
#To calculate the value of e 

err = T_final[1:-1] - T_exp16[1:-1]
print(err)
sum_err = np.sum(err)

err_bar16 = np.abs(sum_err)/grid.ncv

print (err_bar16)

### To calculate the value of P for the second order implicit scheme

In [ ]:
#To generate the plot of e bar vs deltat

from scipy import stats
import numpy as np

x = np.array([dt2, dt4, dt8, dt16, dt32])
print (x)
y = np.array([err_bar2, err_bar4, err_bar8, err_bar16, err_bar32])

print (y)

res = stats.linregress(np.log(x), np.log(y))

#slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
print(res.slope)
print ("Value of P is:", res.slope)

plt.plot(np.log(x), np.log(y), 'o', label='original data')
plt.plot(x, res.intercept + res.slope*x, 'r', label='fitted line')

### Generation of the temperature profile at the centre of the wall for the diverse number of time step for the final time

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig2, ax = plt.subplots()


ax.plot(deltat2, T_cent2, label="N time step = 2")
ax.plot(deltat4, T_cent4, label="N time step = 4")
ax.plot(deltat8, T_cent8, label="N time step = 8")
ax.plot(deltat16, T_cent16, label="N time step = 16")
ax.plot(deltat32, T_cent32, label="N time step = 32")


ax.set_title("T (0,t2) vs delta T for a second order implicit scheme")
ax.set_xlabel("Delta t [s]")
ax.set_ylabel("T [C]")
ax.legend()
plt.show()

## Comparison Analysis between first order implicit and second order implicit schemes

- With the first order implicit scheme for the same number of time step of 32 (delta t was 0.08) the error was 0.60 while with the second order implicit scheme this error was approximately 0.006. Is possible to notice that in case that the error needed to be smaller for more accurate result, the second order implicit scheme will provide a better accuracy without compromision the computational cost. While in the case of the first order scheme, in order to increase the accuracy of the method an increase on the number of time step would be necessary which would be translated to an increase on the computational time of the calculation.

- Is expected that the error on the second order scheme is smaller since the order of accuracy obtained for this scheme is expected to be close to 2 which means that the error will decrease when time step increase but in this case for more accurate results to achieve an specific error, the increase of the time step does not need to be as abrupt as it might need to be on a first order scheme. 

- In the temperature profile changes with time, on the first order scheme the difference of the results obtained between the diverse number of time step implemented has a separation gap higher than the second order where the results for the number of time step of 16 and 32 are more close which means a more consistant result.

- Is possible to conclude that the second order scheme will provide a faster convergence to a smaller error of the transient model than the first order. Despite this, is important to generate the time step independence test to corroborate that the results are accurate in the most effective way without compromising the computational time and computational cost of the calculation.